# English to French translation using the kde4 dataset and t5 tranformer

### Loading the dataset

The dataset is downloaded directly from the hugging face library interface using the 'datasets' library.</br>
Once downloaded the dataset will be present in the cache memory of the notebook and can be accessed for future use.

In [1]:
#Download the kde4 dataset [en-english; fr-french]
from datasets import load_dataset, load_metric
raw_data = load_dataset("kde4", lang1="en", lang2="fr")
raw_data

Using custom data configuration en-fr-lang1=en,lang2=fr


Generating train split:   0%|          | 0/210173 [00:00<?, ? examples/s]

Dataset kde4 downloaded and prepared to C:\Users\Sharanya Manohar\.cache\huggingface\datasets\kde4\en-fr-lang1=en,lang2=fr\0.0.0\243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})

The dataset is available in a single split and needs to be split to create a validation set.

In [2]:
split_data = raw_data["train"].train_test_split(train_size=0.9, seed=20)
split_data

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

'test' key can be renamed as 'validation' for interpretability

In [3]:
split_data["validation"] = split_data.pop("test")

Let us look at one instance of the dataset.

In [4]:
split_data["train"][1]["translation"]

{'en': 'Default to expanded threads',
 'fr': 'Par défaut, développer les fils de discussion'}

### Defining the tokenizer

Pre-defined tokenizer of the pre-trained "t5-small" model is used to tokenize the text in the dataset

In [5]:
model_t5 = "t5-small"
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_t5,use_fast=False)

In [6]:
#t5 transformer models require a prefix indicating the action to be performed on the input provided.
prefix = "translate English to French:" 

### Defining the pre-processing function

In [7]:
#Refer hugging face documentations for language codes
source_language = "en"
target_language = "fr"

max_input_length = 128
max_target_length = 128

def preprocess(instances):
   input = [prefix + i[source_language] for i in instances["translation"]]
   target = [i[target_language] for i in instances["translation"]]
   tokenized_inputs = tokenizer(input, max_length=max_input_length, truncation=True)
   # Setup the tokenizer for target
   with tokenizer.as_target_tokenizer():
       label = tokenizer(target, max_length=max_target_length, truncation=True)
   tokenized_inputs["labels"] = label["input_ids"]
   return tokenized_inputs

In [8]:
#Applying the pre processing on the entire dataset
tokenized_datasets = split_data.map(preprocess, batched=True)

  0%|          | 0/190 [00:00<?, ?ba/s]

  0%|          | 0/22 [00:00<?, ?ba/s]

### Creating subsets of the dataset for faster training

In [10]:
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(1000))

Loading cached shuffled indices for dataset at C:\Users\Sharanya Manohar\.cache\huggingface\datasets\kde4\en-fr-lang1=en,lang2=fr\0.0.0\243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac\cache-dede3008ff233b0c.arrow


### Using the 't5-small' pre-trained model

In [11]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_t5)

In [12]:
batch_size = 16

#defining training attributes
args = Seq2SeqTrainingArguments(
   output_dir="./results",
   evaluation_strategy = "epoch",
   learning_rate=2e-5,
   per_device_train_batch_size=batch_size,
   per_device_eval_batch_size=batch_size,
   weight_decay=0.01,
   save_total_limit=3,
   num_train_epochs=1,
   predict_with_generate=True   
)

In [13]:
#pad inputs and label them
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [15]:
import numpy as np
from datasets import load_metric
metric = load_metric("sacrebleu")
meteor = load_metric('meteor')

#customizing compute_metrics function to display bleu score, mean prediction length and meteor score
def compute_metrics(eval_preds):
   preds, labels = eval_preds
   if isinstance(preds, tuple):
       preds = preds[0]
    
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   # Replacing -100 in the labels as they are not needed and cannot be decoded
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
   
   decoded_preds = [pred.strip() for pred in decoded_preds]
   decoded_labels = [[label.strip()] for label in decoded_labels]
   
   result = metric.compute(predictions=decoded_preds, references=decoded_labels)
   meteor_result = meteor.compute(predictions=decoded_preds, references=decoded_labels)
   prediction_length = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
   
   result = {'bleu' : result['score']}
   result["gen_len"] = np.mean(prediction_length)
   result["meteor"] = meteor_result["meteor"]
   result = {x: round(y, 4) for x, y in result.items()}
   return result

[nltk_data] Downloading package wordnet to C:\Users\Sharanya
[nltk_data]     Manohar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Sharanya
[nltk_data]     Manohar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Sharanya
[nltk_data]     Manohar\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [16]:
#training object with customized parameters
trainer = Seq2SeqTrainer(
   model,
   args,
   train_dataset=train_dataset,
   eval_dataset=eval_dataset,
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics
)


In [17]:
#train model using train function
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: id, translation. If id, translation are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
C:\Users\Sharanya Manohar\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 63


Epoch,Training Loss,Validation Loss,Bleu,Gen Len,Meteor
1,No log,2.037610,10.712800,11.035000,0.156700


The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: id, translation. If id, translation are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=63, training_loss=2.308563716827877, metrics={'train_runtime': 734.8737, 'train_samples_per_second': 1.361, 'train_steps_per_second': 0.086, 'total_flos': 16744318500864.0, 'train_loss': 2.308563716827877, 'epoch': 1.0})